In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchsummary import summary

import matplotlib.pyplot as plt

In [8]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" # it is mps which is gpu but not cuda for mac
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64
Using mps device


In [9]:
class NN(nn.Module): # nn.Module is base class for all models in pytorch
    def __init__(self): # constructor of class
        super().__init__() # super() is used to call the constructor of parent class
        self.flatten  = nn.Flatten() # flatten layer
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [10]:
# Create an instance of the model, including the device it is on.
model = NN().to(device) # to() is used to move the model to the device
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
print(model)


NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [11]:
# define the training loop
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) # get the size of the dataset
    model.train() # set the model to training mode
    for batch, (X, y) in enumerate(dataloader): # iterate over the data, batch is the index of the batch, X is the input and y is the output
        X, y = X.to(device), y.to(device) # move the data to the device
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        # Backpropagation
        loss.backward() # compute the gradient of the loss with respect to the model parameters
        optimizer.step() # adjust the parameters by the gradients collected in the backward pass
        optimizer.zero_grad() # reset the gradients to zero
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X) 
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [12]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() # set the model to evaluation mode
    test_loss, correct = 0, 0
    with torch.no_grad(): # disable gradient computation
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item() # sum up the loss
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() # get the index of the max log-probability
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [13]:
epochs = 5 # number of epochs
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.301414  [   64/60000]
loss: 2.285571  [ 6464/60000]
loss: 2.265106  [12864/60000]
loss: 2.260094  [19264/60000]
loss: 2.253020  [25664/60000]
loss: 2.212368  [32064/60000]
loss: 2.231765  [38464/60000]
loss: 2.191175  [44864/60000]
loss: 2.184275  [51264/60000]
loss: 2.159749  [57664/60000]
Test Error: 
 Accuracy: 46.0%, Avg loss: 2.153062 

Epoch 2
-------------------------------
loss: 2.163518  [   64/60000]
loss: 2.153458  [ 6464/60000]
loss: 2.095387  [12864/60000]
loss: 2.110791  [19264/60000]
loss: 2.069280  [25664/60000]
loss: 1.999846  [32064/60000]
loss: 2.038357  [38464/60000]
loss: 1.956652  [44864/60000]
loss: 1.955865  [51264/60000]
loss: 1.891500  [57664/60000]
Test Error: 
 Accuracy: 58.8%, Avg loss: 1.886781 

Epoch 3
-------------------------------
loss: 1.920731  [   64/60000]
loss: 1.892336  [ 6464/60000]
loss: 1.769281  [12864/60000]
loss: 1.807738  [19264/60000]
loss: 1.697378  [25664/60000]
loss: 1.643645  [32064/600

In [14]:
# save model state_dict
torch.save(model.state_dict(), "model_state_dict.pth")
# save the entire model
torch.save(model, "model_entire.pth")


In [3]:
class NN(nn.Module): # nn.Module is base class for all models in pytorch
    def __init__(self): # constructor of class
        super().__init__() # super() is used to call the constructor of parent class
        self.flatten  = nn.Flatten() # flatten layer
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
# load model state_dict
model = NN()
model.load_state_dict(torch.load("model_state_dict.pth"))

/var/folders/g0/phgwvz7x3sz18gfzz4zv9rq40000gn/T/ipykernel_75049/2299125878.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model_state

<All keys matched successfully>

In [5]:
# load the entire model
model = torch.load("model_entire.pth")

/var/folders/g0/phgwvz7x3sz18gfzz4zv9rq40000gn/T/ipykernel_75049/2511868538.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("model_entire.pth")


the difference between the two is that the state_dict only contains the model’s weights, while the entire model contains the weights, as well as the model’s architecture and other state_dict values. But we need to ensure that the model class is defined as it was when the model was saved.